<a href="https://colab.research.google.com/github/HRashidLiaquat/lessons-learned/blob/main/Fruits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Project Title: Fruit Classification with Convolutional Neural Network**


**Project Workflow**
1.  Import important libraries
2.  Get data ready in Kaggle.
3.  Preparing My Images
4.  Loading training images
5.  Create Data Loaders
6.  Build A model
7.  Model training
8.  Model Testing (Training Loop)
9.  Model Evaluation
10.  Testing with new data point
11.  Save model

**Import important libraries**

In [2]:
from pathlib import Path
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim
